In [1]:
# 웹 드라이버 및 url 설정
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import os
import requests

C:\Users\bestl\anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
directory = 'test_final_1'
if not os.path.exists(directory):
    os.makedirs(directory)

In [4]:
# 이미지 있는지 확인 (함수 필요하다면)

def check_img_tag(html):
  soup = BeautifulSoup(html, 'html.parser')
  img_tags = soup.find_all('img')
  if img_tags:
      return True
  else:
      return False
    
def get_and_save_img(date):
    # 이미지 추출
    find_img = driver.find_element(By.CLASS_NAME, 'view-con')
    img = find_img.find_elements(By.TAG_NAME, 'img')

    # 이미지가 없는 경우 None 반환
    if not img:
        return None

    img_src = img[0].get_attribute('src')

    # 디렉토리 만들기 -> 폴더 이름 수정 원할 시 directory명 변경
    directory = 'test_final_1'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # 이미지 파일 이름 생성
    filename = f'{directory}/{date}.jpg'
    count = 1
    while os.path.exists(filename):
        filename = f'{directory}/{date}_{count}.jpg'
        count += 1

    # 이미지 다운로드
    response = requests.get(img_src)

    # 이미지를 로컬 파일로 저장
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"{date} 이미지를 성공적으로 저장했습니다.")
    else:
        print(f"{date} 이미지를 다운로드할 수 없습니다.")

    return img_src  # 이미지가 있는 경우 이미지 소스 반환

# 텍스트 함수

## 글내용 스크랩 함수
def get_content():
    # 페이지의 소스 가져오기
    html_source = driver.page_source

    # BeautifulSoup 객체 생성하여 HTML 파싱
    soup = BeautifulSoup(html_source, 'html.parser')

    # CSS 선택자를 사용하여 내용 추출
    content = soup.select_one("div.view-con").get_text(strip=False) if soup.select_one("div.view-con") else ''

    # 텍스트 정리
    content = content.replace("\t", "").replace('\xa0', '')

    # 글내용 반환
    return content


## 제목, 날짜 스크랩 함수.
def get_titleDate():
    # 페이지의 소스 가져오기
    html_source = driver.page_source

    # BeautifulSoup 객체 생성하여 HTML 파싱
    soup = BeautifulSoup(html_source, 'html.parser')

    # CSS 선택자를 사용하여 날짜, 제목 추출
    title = soup.select_one("div.board-view-info div.view-info h2").get_text(strip=True) if soup.select_one("div.board-view-info div.view-info h2") else ''
    date = soup.select_one("div.view-info div.view-detail dl.write dd").get_text(strip=True) if soup.select_one("div.view-info div.view-detail dl.write dd") else ''

    # 텍스트 정리
    title = title.replace("\t", "").replace('\xa0', '')

    # 제목, 날짜 반환 : ('제목', '날짜') 튜플 형태
    return title, date



In [5]:
# Chrome WebDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')

# Chrome WebDriver 초기화 및 웹 페이지 열기
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(5)

# 학사공지 홈페이지 접속
# url = "https://www.hufs.ac.kr/hufs/11282/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGaHVmcyUyRjIxODElMkZhcnRjbExpc3QuZG8lM0Y%3D"

# 장학공지 홈페이지 접속
# 이미지 들어오나 확인
url = "https://www.hufs.ac.kr/hufs/11283/subview.do"

#채용
#url = "https://www.hufs.ac.kr/hufs/11284/subview.do"
driver.get(url)

# csv에 담아 출력할 DataFrame 생성
import pandas as pd
df_crawling = pd.DataFrame(columns = ['index', 'date', 'title', 'content'])


# index 수집
# Selenium으로 페이지의 HTML 소스 가져오기
html = driver.page_source

# BeautifulSoup 객체 생성하여 HTML 파싱
soup = BeautifulSoup(html, 'html.parser')

# CSS 선택자로 인덱스 추출
elements = soup.select("div._fnctWrap table tbody tr:not(.notice) td.td-num")
index_texts = [element.get_text() for element in elements]

# 추출한 인덱스 형변환
index = int(index_texts[0]) if index_texts else None


# 첫번째 공지 글 클릭
# CSS 선택자를 사용하여 공지태그가 아닌 첫 번째 글의 <a> 태그를 찾기
first_page = driver.find_element(By.CSS_SELECTOR, "div.scroll-table tbody tr:not(.notice) a")

# 클릭
first_page.click()

In [6]:
while True:
    try:
        title = get_titleDate()[0]
        date = get_titleDate()[1]
        img_flag = check_img_tag(date)
        if img_flag:
            get_and_save_img(date)
        content = get_content()

        # 잘 추출되었는지 확인
        print(f"Index: {index}")
        print(f"Date: {date}")
        print(f"Title: {title}")
        print(f"Content: {content}")

        # 데이터프레임에 추가
        new_row = {'index': index, 'date': date, 'title': title, 'content': content}
        df_crawling.loc[len(df_crawling)] = new_row

        # 인덱스 갱신
        index -= 1

        # 이전글 클릭
        # CSS 선택자를 사용하여 이전글의 <a> 태그를 찾기
        previous_page = driver.find_element(By.CSS_SELECTOR, "main.contents div._fnctWrap div.view-navi dl:first-child dd a")

        # 클릭
        previous_page.click()
        time.sleep(2)

    except Exception as e:
        print(e)
        break

Index: 1260
Date: 2024.03.28
Title: [공통][교내]2024-1 윤강로 장학금 장학생 모집 안내(~4/17)
Content: 
2024-1윤강로 장학금 장학생 모집 안내

아래와 같이2024-1학기 윤강로(인도어77)장학금 장학생 모집을 하오니 해당되는 재학생들의 많은 지원 바랍니다.

-아 래-


1.장학금명:윤강로 장학금

2.선발인원 및 지원금액:총2명(양 캠퍼스 각1명),등록금 전액
*타 장학금 수혜내역이 있는 학생의 경우,선발 후 윤강로 장학금 수혜액과 합쳐서 등록금 범위 초과 수혜 불가능함.

3.지원자격
가. 2024-1학기 서울 및 글로벌캠퍼스 재학생(2학기 이상7학기 이하 재학생)
나.직전학기 평점4.0/4.5이상인 학생
다.영어능력평가시험(TOEIC, TOEFL, FLEX)성적소지자
라.가계곤란자(24-1학기 한국장학재단 학자금지원구간으로 심사)
*선발시 기부자 감사편지(자필)제출 필수
*장경자 장학금 중복신청 불가(중복신청 시 탈락)

4.세부일정
가.지원기간:2024. 04. 11(목) ~ 4. 17(수) 17:00까지
나.선발: 5월 초(예정)
다.선발확인: HUFS Ability로그인 후 장학금 신청내역에서 확인

5.제출서류
가.장학금 지원서1부(HUFS Ability다운로드)
나.자기소개서(자유형식, A4 1장)
다.영어능력평가시험(TOEIC, TOEFL, FLEX중 택1)성적증명서1부
라.개인정보 수집.이용 동의서1부(HUFS Ability다운로드)
*반드시PDF 1개 파일로 제출(파일명:윤강로장학금 지원서(학번,이름).pdf )

6.신청방법:온라인 신청(HUFS Ability-학생핵심역량통합시스템)→로그인(학번/비번)→교과/비교과→장학신청 목록→해당 장학금

7.지급방법:계좌 지급(종합정보시스템 등록 계좌로 지급 예정)
※종합정보시스템 로그인→등록/장학→계좌정보 입력에 반드시 본인명의 계좌 입력 요망

8.유의사항:대출금 및 기수혜받은 장학금액의 합계가 해당 학기 등록금액을 초과할 경우 대출금을 우

KeyboardInterrupt: 

In [ ]:
# Selenium WebDriver 종료
driver.quit()

In [7]:
# 내 경로로 설정
import os

# 저장할 디렉토리 경로
save_directory = os.path.join(os.path.expanduser('~'), 'Desktop')

# 디렉토리가 없다면 생성
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# CSV 파일을 해당 디렉토리에 저장
df_crawling.to_csv(os.path.join(save_directory, 'noti_janghak.csv'), encoding="UTF-8", index=False)


In [8]:
import pandas as pd
file = "noti_janghak.csv"
ct = pd.read_csv(file, encoding = 'UTF-8')
ct

,index,date,title,content
0,1260,2024.03.28,[공통][교내]2024-1 윤강로 장학금 장학생 모집 안내(~4/17),\n2024-1윤강로 장학금 장학생 모집 안내\n\n아래와 같이2024-1학기 윤강...
1,1259,2024.03.28,[공통][교내]2024-1 故장경자 장학금 장학생 모집 안내(~4/17),\n2024-1故장경자 장학금 장학생 모집 안내\n\n아래와 같이2024-1학기故장...
